[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uzusio/whisper_tool.ipynb/blob/main/whisper_tool.ipynb)
# 音声文字起こしツール for whisper


---


## 概要
OpenAIの[whisper](https://github.com/openai/whisper)を使用した音声文字起こしツールです。
Google Colabの無料環境で利用可能です。

---


## 使い方

1.   編集->ノートブックの設定->ハードウェア アクセラレータ から「GPU」を選択してください（※初回以降は不要です）
2.   Googleドライブに「whisper」というフォルダを作成してください（※初回以降は不要です）
3.   一番目のセルを実行して、Googleドライブをマウントしてください
4.   二番目のセルに設定を記述して、実行してください
    1.   DIR_PATH: 文字起こしをする音声ファイルを配置するディレクトリです。文字起こしされたテキストファイルも同ディレクトリに出力されます。デフォルトではドライブ上の「whisper」フォルダになります。
    2.   INPUT_FILE: 文字起こしする音声ファイルのファイル名です。 
    3.   MODEL_TYPE： 文字起こしの精度を選びます。tiny<base<small<medium<largeの順に精度が上がります。※ダウンロードするモデルのファイルサイズも大きくなります。メモリエラーが起きたらモデルを１段階小さくしてください。
    4.   srt_output: 字幕ファイル生成に対応しました。出力ファイルを字幕ファイル形式にしたい場合はチェックを入れてください。
5.以降のセルを最後まで実行してください。ランタイム->以降のセルを実行でもOKです。

### おまけ
マイクからの入力機能も用意してみました。

### 翻訳機能を追加（new!）
Google翻訳とPapago翻訳を追加しました。DeepLも追加予定です。






In [ ]:
#@title ## Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

In [64]:
#@title ## 初期設定
#@markdown ### 基本設定:
#@markdown ・ファイルを配置するディレクトリを指定
DIR_PATH="/content/drive/MyDrive/whisper" #@param {type:"string"}
#@markdown ・入力ファイル名
INPUT_FILE="sample.mp4" #@param {type:"string"}
#@markdown ・利用するモデルサイズを選択
MODEL_TYPE="large" #@param ["tiny", "base","small","medium","large"] {type:"string"}

#@markdown ---


#@markdown ### 翻訳設定:

#@markdown ・翻訳機能を有効にする場合下にチェック
translate_option=True #@param {type: "boolean"}


#@markdown ・翻訳エンジンを指定
TRANSLATOR = 'GoogleTranslator' #@param ["GoogleTranslator", "PapagoTranslator"]

#@markdown ・翻訳先言語を指定（翻訳元言語は自動判定されます）
TRANS_TO  = 'ja' #@param  ["ja","en","ko","zh-CN","zh-TW","es","fr","vi","th","id"]

#@markdown ・「CLIENT_ID / SECRET_KEY」を指定（PapagoTranslatorを使用する場合に必要）
CLIENT_ID = '' #@param {type:"string"}
SECRET_KEY = '' #@param {type:"string"}

#@markdown ---


#@markdown ### 字幕ファイル設定（動画編集用）:
#@markdown ・字幕ファイル(.srt)を出力したい場合下にチェック
srt_output=True #@param {type: "boolean"}
#@markdown ・字幕ファイル作成時、自動改行させる場合下にチェック
line_option=True #@param {type: "boolean"}



In [ ]:
#@title ## 必要ライブラリのインストール
!pip install git+https://github.com/openai/whisper.git
!pip install srt
!pip install deep-translator 
!pip install papago

In [ ]:
#@title ## モデルのロード
import whisper

model = whisper.load_model(MODEL_TYPE)

100%|█████████████████████████████████████| 2.87G/2.87G [00:36<00:00, 83.9MiB/s]


In [49]:
#@title ## 文字起こし実行
file_path = DIR_PATH + "/" + INPUT_FILE

# 推論
result = model.transcribe(file_path)


In [67]:
#@title ## 結果をGoogleドライブに保存
from datetime import timedelta
from srt import Subtitle
import srt
from deep_translator import GoogleTranslator
from papago import Translator
import copy

# 翻訳実行
if translate_option:
  result_translated = copy.deepcopy(result)

  if TRANSLATOR == 'GoogleTranslator':
    for i,segment in enumerate(result["segments"]):
      result_translated['segments'][i]['text'] = GoogleTranslator(source='auto',target=TRANS_TO).translate(segment['text'])

  elif TRANSLATOR == 'PapagoTranslator':
    translator = Translator(CLIENT_ID, SECRET_KEY)
    for i,segment in enumerate(result["segments"]):
      result_translated['segments'][i]['text'] = translator.translate(text=segment['text'], target=TRANS_TO).text


def add_line(s):
    new_s = s
    s_count = len(s)
    s_max_count = 15
    if s_count >= s_max_count:
        if (s_count - s_max_count) >= 3:
            # 18文字以上であれば、15文字で改行する
            new_s = s[:s_max_count] + "\n" + s[s_max_count:]
 
    return new_s

def result2subs(result):
  subs = []

  for segment in result["segments"]:
    index = segment["id"] + 1
    start = segment["start"]
    end = segment["end"]
    text = add_line(segment["text"]) if line_option else segment["text"]
    sub = Subtitle(index=1, start=timedelta(seconds=timedelta(seconds=start).seconds,
                                            microseconds=timedelta(seconds=start).microseconds),
                   end=timedelta(seconds=timedelta(seconds=end).seconds,
                                 microseconds=timedelta(seconds=end).microseconds), content=text, proprietary='')
 
    subs.append(sub)
  return subs

# 字幕出力
if srt_output:
  with open(f"{file_path}.srt", "w") as f:
    f.write(srt.compose(result2subs(result)))
  
  if translate_option:
    with open(f"{file_path}_{TRANS_TO}.srt", "w") as f:
      f.write(srt.compose(result2subs(result_translated)))

else:
  with open(f"{file_path}.txt", "w") as f:
    f.write(f"▼ Transcription of {INPUT_FILE}\n")
    for segment in result["segments"]:
      f.write(str(segment["id"]) + " " + segment["text"] + "\n")



In [ ]:
#@title ## 結果をテーブル表示で見る
import pandas as pd
pd.DataFrame(result["segments"])[["start", "end", "text"]]
pd.DataFrame(result_translated["segments"])[["start", "end", "text"]]

# マイクから入力機能

In [ ]:
#@title ## 必要ライブラリのインストール
!pip install transformers
!pip install torch
!pip install gradio

In [ ]:
#@title ## マイクから入力
from transformers import pipeline
import whisper
import gradio as gr

if not model:
  model = whisper.load_model(MODEL_TYPE)

p = pipeline("automatic-speech-recognition")

def transcribe(audio):
    text = model.transcribe(audio)
    return text

gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text").launch()
